## Data load & Data preprocessing

* **The Energy Efficiency Dataset** from the UCI Machine Learning Repository contains information related to the thermal performance of buildings.
* It includes **8 predictor variables** describing physical and design parameters such as surface area, wall and roof area, building orientation, and glazing area.
* The dataset provides two response variables: Heating Load and Cooling Load, representing the building’s energy demand for heating and cooling.
* In this project, a predictive model (regression) was developed to estimate the **Heating Load** based on the given building features.

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StandardScaler, PolynomialExpansion, Interaction
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.sql.functions import col, when, count
from pyspark.ml.feature import VectorAssembler as VA2
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName("EnergyEfficiencyRegression").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 15:03:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_pandas = pd.read_csv("ENB2012_data.csv")

# Rename columns
df_pandas.columns = ["Relative_Compactness",        # X1
                     "Surface_Area",                # X2
                     "Wall_Area",                   # X3
                     "Roof_Area",                   # X4
                     "Overall_Height",              # X5
                     "Orientation",                 # X6
                     "Glazing_Area",                # X7
                     "Glazing_Area_Distribution",  # X8
                     "Heating_Load",                # Y1
                     "Cooling_Load"                 # Y2
                    ]

# Pandas -> PySpark DataFrame
df = spark.createDataFrame(df_pandas)

In [4]:
# heck dataset information
df.printSchema()
print(df.count())

root
 |-- Relative_Compactness: double (nullable = true)
 |-- Surface_Area: double (nullable = true)
 |-- Wall_Area: double (nullable = true)
 |-- Roof_Area: double (nullable = true)
 |-- Overall_Height: double (nullable = true)
 |-- Orientation: double (nullable = true)
 |-- Glazing_Area: double (nullable = true)
 |-- Glazing_Area_Distribution: double (nullable = true)
 |-- Heating_Load: double (nullable = true)
 |-- Cooling_Load: double (nullable = true)



1296


* For simplicity, missing values in the dataset were removed prior to model training.
* Alternatively, if desired, various imputation methods (e.g., mean, median, or interpolation) can be applied to handle missing data.

In [ ]:
df = df.dropna()
print(df.count())

768


In [ ]:
# Split data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Create features for transformation
feature_cols = ["Surface_Area", "Wall_Area", "Roof_Area", "Overall_Height", "Glazing_Area",
                "Orientation", "Glazing_Area_Distribution"]
feature_poly = ["Surface_Area", "Wall_Area", "Roof_Area", "Glazing_Area"]
feature_inter = ["Orientation", "Glazing_Area"]

## Model fitting

Since the response value in this project (Heating Load) is a continuous numerical value, a regression approach was required to predict it effectively.  
To address this, I selected and compared three different regression models: Multiple Linear Regression, Regression Tree, and Random Forests.  
* **Multiple Linear Regression model**
  * A statistical method that models the linear relationship between multiple predictor variables and a continuous response value.
  * It assumes that the response value is a linear combination of the predictor variables.  

* **Regression Tree model**
  - A decision tree-based model that splits the data into regions using feature thresholds to minimize prediction error.
  - It captures non-linear relationships and is easy to interpret.  

* **Random Forests model**
  - An ensemble method based on bagging (bootstrap aggregating)
  - It constructs multiple decision trees using random subsets of the training data (with replacement).
  - For each tree, a random subset of predictor variables is selected at each split, enhancing model diversity.
  - It improves accuracy and reduces overfitting compared to a single decision tree.

#### Metric
* Metric used: Root Mean Squared Error (RMSE)
* RMSE measures the square root of the average squared differences between predicted and actual values.
* It penalizes large errors more than small ones, making it sensitive to outliers.
* RMSE is widely used in regression tasks and provides results in the same unit as the target variable.

In this section, pipelines were constructed for each regression model, and cross-validation was performed to identify the best-performing model.  
The best model was then retrained using the entire dataset to finalize the learning process.

### 1. Multiple Linear Regression model

In [22]:
# Define transformations
# Feature vectorization
assembler_lr = VectorAssembler(inputCols=feature_cols, outputCol="assembled_features")
# Vectorization → Normalization
scaler_lr = StandardScaler(inputCol="assembled_features", outputCol="features", withMean=True, withStd=True)

In [13]:
# Create model object
lr = LinearRegression(featuresCol="features", labelCol="Heating_Load")

# Create parameter grid
paramGrid_lr = ParamGridBuilder()  \
    .addGrid(lr.regParam, [0, 0.01, 0.04, 0.06, 0.1])  \
    .addGrid(lr.elasticNetParam, [0, 0.5, 0.8, 1])  \
    .build()

In [14]:
# Build pipeline
pipeline_lr = Pipeline(stages = [assembler_lr, scaler_lr, lr]) 

In [15]:
# Create cross-validation object
crossval_lr = CrossValidator(estimator = pipeline_lr, 
                             estimatorParamMaps = paramGrid_lr,
                             evaluator = RegressionEvaluator(labelCol="Heating_Load", metricName='rmse'),
                             numFolds = 5) 

In [16]:
lr_cv = crossval_lr.fit(train_data)

25/04/08 15:03:46 WARN Instrumentation: [db68090b] regParam is zero, which might cause numerical instability and overfitting.
25/04/08 15:03:46 WARN Instrumentation: [db68090b] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
25/04/08 15:03:50 WARN Instrumentation: [da1d07fd] regParam is zero, which might cause numerical instability and overfitting.
25/04/08 15:03:50 WARN Instrumentation: [da1d07fd] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
25/04/08 15:03:51 WARN Instrumentation: [207ea4a8] regParam is zero, which might cause numerical instability and overfitting.
25/04/08 15:03:52 WARN Instrumentation: [a15caecc] regParam is zero, which might cause numerical instability and overfitting.
25/04/08 15:04:04 WARN Instrumentation: [18df9b33] regParam is zero, which might cause numerical instability and overfitting.
25/04/08 15:04:04 WARN Instrumentation: [18df9b33] Cholesky solver failed due to s

In [17]:
lr_list = []
for i in range(len(paramGrid_lr)):
    lr_list.append([lr_cv.avgMetrics[i], paramGrid_lr[i].values()])
lr_list

[[2.9103895221806093, dict_values([0.0, 0.0])],
 [2.910389522180748, dict_values([0.0, 0.5])],
 [2.9103895221605622, dict_values([0.0, 0.8])],
 [2.910389522160519, dict_values([0.0, 1.0])],
 [2.9105541822095935, dict_values([0.01, 0.0])],
 [2.9109813782581986, dict_values([0.01, 0.5])],
 [2.9111042830531053, dict_values([0.01, 0.8])],
 [2.911354879179608, dict_values([0.01, 1.0])],
 [2.913938756007699, dict_values([0.04, 0.0])],
 [2.9165763914472778, dict_values([0.04, 0.5])],
 [2.9177139469363405, dict_values([0.04, 0.8])],
 [2.9176441309951087, dict_values([0.04, 1.0])],
 [2.917919115974555, dict_values([0.06, 0.0])],
 [2.943560851440754, dict_values([0.06, 0.5])],
 [2.917737956079703, dict_values([0.06, 0.8])],
 [2.9291954360425416, dict_values([0.06, 1.0])],
 [2.9283364363744133, dict_values([0.1, 0.0])],
 [2.9177106317165924, dict_values([0.1, 0.5])],
 [2.9186097108722544, dict_values([0.1, 0.8])],
 [2.9196404139598595, dict_values([0.1, 1.0])]]

In [16]:
# Extract best model parameters
best_index_lr = lr_cv.avgMetrics.index(min(lr_cv.avgMetrics))
best_params_lr = paramGrid_lr[best_index_lr]

In [17]:
# Train best model on the full dataset
lr_final = pipeline_lr.copy(best_params_lr).fit(train_data)

25/04/08 15:07:37 WARN Instrumentation: [a324b2c2] regParam is zero, which might cause numerical instability and overfitting.


### 2. Regression Tree model

In [18]:
# Define transformations
# Feature vectorization
assembler_dt = VectorAssembler(inputCols=feature_cols, outputCol="features")

Note that normalization was not applied to the tree-based models are not sensitive to feature scaling.  
These models split data based on feature thresholds and do not rely on distance-based calculations, making normalization unnecessary.

In [19]:
# Create Tree model ojbect
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Heating_Load")

# Create parameter grid
paramGrid_dt = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [3, 5, 10]) \
    .addGrid(dt.minInstancesPerNode, [1, 2, 4]) \
    .build()

In [20]:
# Build pipeline
pipeline_dt = Pipeline(stages=[assembler_dt, dt])

In [21]:
# Create cross-validation object
crossval_dt = CrossValidator(
    estimator=pipeline_dt,
    estimatorParamMaps=paramGrid_dt,
    evaluator=RegressionEvaluator(labelCol="Heating_Load", metricName="rmse"),
    numFolds=5
)

In [22]:
dt_cv = crossval_dt.fit(train_data)

In [23]:
dt_list = []
for i in range(len(paramGrid_dt)):
    dt_list.append([dt_cv.avgMetrics[i], paramGrid_dt[i].values()])
dt_list

[[2.434899777059463, dict_values([3, 1])],
 [2.434899777059463, dict_values([3, 2])],
 [2.434899777059463, dict_values([3, 4])],
 [1.136697260472595, dict_values([5, 1])],
 [1.1601286674276732, dict_values([5, 2])],
 [1.2689179779053141, dict_values([5, 4])],
 [0.6482622977146174, dict_values([10, 1])],
 [0.6971580927109831, dict_values([10, 2])],
 [0.8099170243838809, dict_values([10, 4])]]

In [26]:
# Extract best model parameters
best_index_dt = dt_cv.avgMetrics.index(min(dt_cv.avgMetrics))
best_params_dt = paramGrid_dt[best_index_dt]

In [27]:
# Train best model on the full dataset
dt_final = pipeline_dt.copy(best_params_dt).fit(train_data)

### 3. Random Forests model 

In [28]:
# Define transformations
# Feature vectorization -1
assembler_base = VectorAssembler(inputCols=feature_cols, outputCol="base_features")

# Feature vectorization -2
assembler_poly = VectorAssembler(inputCols=feature_poly, outputCol="poly_input")
# Vectorization → Add polynomial features
poly = PolynomialExpansion(inputCol="poly_input", outputCol="poly_features", degree=2)

# Feature vectorization -3
assembler_f1 = VectorAssembler(inputCols=["Wall_Area"], outputCol="vec1")
assembler_f2 = VectorAssembler(inputCols=["Glazing_Area"], outputCol="vec2")
# Vectorization → Add interaction features
interaction = Interaction(inputCols=["vec1", "vec2"], outputCol="interaction_feature")

In [29]:
# Combine all features (polynomial + interaction) → Vectorization
final_features = VectorAssembler(inputCols=["base_features", "poly_features", "interaction_feature"],
                                 outputCol="assembled_features"
                                )

In [30]:
# Vectorization → Normalization
scaler = StandardScaler(inputCol="assembled_features", outputCol="features", withMean=True, withStd=True)

In [31]:
# Create model ojbect
rf = RandomForestRegressor(featuresCol="features", labelCol="Heating_Load")

# Create parameter grid
paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 30, 50]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

In [32]:
# Build pipeline
pipeline_rf = Pipeline(stages = [assembler_base, assembler_poly, poly, assembler_f1, assembler_f2, interaction, final_features, scaler, rf]) 

In [33]:
crossval_rf = CrossValidator(
    estimator=pipeline_rf,
    estimatorParamMaps=paramGrid_rf,
    evaluator=RegressionEvaluator(labelCol="Heating_Load", metricName="rmse"),
    numFolds=5
)

In [34]:
rf_cv = crossval_rf.fit(train_data)

25/04/08 15:10:00 WARN DAGScheduler: Broadcasting large task binary with size 1100.5 KiB
25/04/08 15:10:09 WARN DAGScheduler: Broadcasting large task binary with size 1100.5 KiB
25/04/08 15:10:10 WARN DAGScheduler: Broadcasting large task binary with size 1162.4 KiB
25/04/08 15:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1043.8 KiB
25/04/08 15:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1265.5 KiB
25/04/08 15:10:24 WARN DAGScheduler: Broadcasting large task binary with size 1526.9 KiB
25/04/08 15:10:26 WARN DAGScheduler: Broadcasting large task binary with size 1703.0 KiB
25/04/08 15:10:33 WARN DAGScheduler: Broadcasting large task binary with size 1265.5 KiB
25/04/08 15:10:35 WARN DAGScheduler: Broadcasting large task binary with size 1526.9 KiB
25/04/08 15:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1703.0 KiB
25/04/08 15:10:38 WARN DAGScheduler: Broadcasting large task binary with size 1799.2 KiB
25/04/08 15:10:39 WAR

In [35]:
rf_list = []
for i in range(len(paramGrid_rf)):
    rf_list.append([rf_cv.avgMetrics[i], paramGrid_rf[i].values()])
rf_list

[[0.9726035535459465, dict_values([10, 5])],
 [0.640050100271086, dict_values([10, 10])],
 [0.6406749199863866, dict_values([10, 15])],
 [0.9001581177754586, dict_values([30, 5])],
 [0.6106290766809306, dict_values([30, 10])],
 [0.6105919325883014, dict_values([30, 15])],
 [0.8961704139505494, dict_values([50, 5])],
 [0.5927476104743807, dict_values([50, 10])],
 [0.5925069611108009, dict_values([50, 15])]]

In [36]:
# Extract best model parameters
best_index_rf = rf_cv.avgMetrics.index(min(rf_cv.avgMetrics))
best_params_rf = paramGrid_rf[best_index_rf]

In [37]:
# Train best model on the full dataset
rf_final = pipeline_rf.copy(best_params_rf).fit(train_data)

25/04/08 15:16:44 WARN DAGScheduler: Broadcasting large task binary with size 1278.1 KiB
25/04/08 15:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1572.8 KiB
25/04/08 15:16:49 WARN DAGScheduler: Broadcasting large task binary with size 1783.4 KiB
25/04/08 15:16:50 WARN DAGScheduler: Broadcasting large task binary with size 1914.6 KiB
25/04/08 15:16:52 WARN DAGScheduler: Broadcasting large task binary with size 1944.8 KiB
25/04/08 15:16:53 WARN DAGScheduler: Broadcasting large task binary with size 1432.9 KiB


## Model Testing  
In this section, final predictions were made using the trained models for each regression approach.

In [38]:
# Create a common evaluator
evaluator = RegressionEvaluator(
    labelCol="Heating_Load",
    predictionCol="prediction",
    metricName="rmse"
)

In [39]:
# Make predictions using each model
pred_lr = lr_final.transform(test_data)
pred_dt = dt_final.transform(test_data)
pred_rf = rf_final.transform(test_data)

In [40]:
# Compute RMSE for each model
rmse_lr = evaluator.evaluate(pred_lr)
rmse_dt = evaluator.evaluate(pred_dt)
rmse_rf = evaluator.evaluate(pred_rf)

print(rmse_lr, rmse_dt, rmse_rf)

3.3533647548048124 0.57934717838548 0.49263459516465913


## Conclusion

Among the three models evaluated, the Random Forest model with added polynomial and interaction features demonstrated the best performance, achieving the lowest RMSE of 0.49.
This indicates that the model was able to capture complex, non-linear relationships between building design features and heating load more effectively than the other models.
In contrast, the Linear Regression model, which assumes linearity, showed the highest RMSE (3.35), suggesting it may not fully capture the underlying patterns in the data.
The Regression Tree performed better than the linear model (RMSE 0.58), but the ensemble nature of Random Forest, combined with feature expansion, provided further improvements by reducing overfitting and increasing robustness.